# HSI Surface Mapping
Segment a synthetic HSI cube, extract ROI spectra, and run RQ.

In [ ]:

import numpy as np
import pandas as pd

from foodspec.protocol_engine import ProtocolConfig, ProtocolRunner
from foodspec.spectral_dataset import HyperspectralDataset

npz = np.load('examples/data/hsi_synthetic.npz')
cube = npz['cube']
wn = npz['wavenumbers']
meta = pd.DataFrame({'pixel_id': np.arange(cube.shape[0]*cube.shape[1])})
hsi = HyperspectralDataset.from_cube(cube, wn, meta)
proto = {
    'name': 'hsi_demo',
    'steps': [
        {"type":"hsi_segment","params":{"method":"kmeans","n_clusters":2}},
        {"type":"hsi_roi_to_1d","params":{"peaks":[{"name":"I_1000","wavenumber":1000.0},{"name":"I_1100","wavenumber":1100.0}],"ratios":[{"name":"1000/1100","numerator":"I_1000","denominator":"I_1100"}],"run_rq":True,"oil_col":"roi_label"}},
        {"type":"output","params":{"output_dir":"runs_notebook"}}
    ]
}
cfg = ProtocolConfig.from_dict(proto)
runner = ProtocolRunner(cfg)
res = runner.run([hsi])
print('Run dir:', res.run_dir)
if 'hsi_roi_peaks' in res.tables:
    print(res.tables['hsi_roi_peaks'].head())
